In [2]:
import spiceypy as spice
import pandas as pd
import numpy as np
import glob

In [3]:
def explore_ek():
    path = "/home/dominik/dev/tudat-bundle/spice/lro/data"
    rows = []
    
    ek_idx_start = 10
    spice.furnsh(f"{path}/lsk/naif0012.tls")
    for file in sorted(glob.glob(f"{path}/ek/lroevnt_2010*.bes"))[ek_idx_start:ek_idx_start+20]:
        spice.furnsh(file)
    
        
    for i in range(spice.ekccnt("LRO_EVENTS")):
        col_name, col_attr = spice.ekcii("LRO_EVENTS", i)
        print(col_name, col_attr)
        
    query = f"SELECT EVT_TIME, SC_TASK, SC_MSG_ID, STOL_PROC_NAME, DESCRIPTION FROM LRO_EVENTS " \
              "WHERE EVT_TIME BETWEEN '2010 JUN 25' AND '2010 JUN 27' " \
              "ORDER BY EVT_TIME"
            
    n_rows, _, _ = spice.ekfind(query)
    print(n_rows)
    
    rows = []
    
    tasks = set()
    for i in range(0, min(n_rows, 5)):
    # for i in range(n_rows-1000, n_rows):
    # for i in range(n_rows):
        evt_time, _ = spice.etcal(spice.ekgd(0, i, 1))
        sc_task, _ = spice.ekgc(1, i, 1)
        sc_msg_id, _ = spice.ekgc(2, i, 1)
        stol_proc_name, _ = spice.ekgc(3, i, 1)
        description, _ = spice.ekgc(4, i, 0)
        # if sc_task == " AC_APP":
        #     print(sc_task, description, evt_time)
        print(evt_time, sc_task, sc_msg_id, stol_proc_name, description)
        tasks.add(sc_task)
    # print(tasks)
    
    spice.kclear()
        
# explore_ek()

EVT_TIME <SpiceEKAttDsc cclass = 2, dtype = 3, strlen = 1, size = 1, indexd = True, nullok = False >
SC_TASK <SpiceEKAttDsc cclass = 3, dtype = 0, strlen = -1, size = 1, indexd = True, nullok = True >
SC_MSG_ID <SpiceEKAttDsc cclass = 3, dtype = 0, strlen = -1, size = 1, indexd = True, nullok = True >
STOL_PROC_NAME <SpiceEKAttDsc cclass = 3, dtype = 0, strlen = -1, size = 1, indexd = True, nullok = True >
DESCRIPTION <SpiceEKAttDsc cclass = 3, dtype = 0, strlen = -1, size = 1, indexd = True, nullok = False >
391
2010 JUN 25 00:04:05.210  SC_APP ID:0024  RTS Number 173 Started.
2010 JUN 25 00:04:26.210  SC_APP ID:0027  Relative Time Sequence #173 Execution Completed.
2010 JUN 25 00:18:14.211  SC_APP ID:0024  RTS Number 170 Started.
2010 JUN 25 00:18:20.209  SC_APP ID:0027  Relative Time Sequence #170 Execution Completed.
2010 JUN 25 01:56:19.209  SC_APP ID:0024  RTS Number 173 Started.


In [4]:
def build_ek_dataframe():
    path = "/home/dominik/dev/tudat-bundle/spice/lro/data"
    rows = []
    
    files = sorted(glob.glob(f"{path}/ek/lroevnt_2010*.bes"))
    file_idx = 0
    
    while file_idx < len(files):
        spice.furnsh(f"{path}/lsk/naif0012.tls")
        for file in files[file_idx:file_idx+20]:
            spice.furnsh(file)

        query = f"SELECT EVT_TIME, SC_TASK, DESCRIPTION FROM LRO_EVENTS ORDER BY EVT_TIME"

        n_rows, _, _ = spice.ekfind(query)

        for i in range(n_rows):
            evt_time, _ = spice.etcal(spice.ekgd(0, i, 1))
            sc_task, _ = spice.ekgc(1, i, 1)
            description, _ = spice.ekgc(2, i, 0)
            rows.append((evt_time, sc_task, description))

        spice.kclear()
        
        file_idx += 20
        
    df = pd.DataFrame(rows, columns=["t", "task", "desc"])
    df["t"] = pd.to_datetime(df["t"])
    df = df.set_index("t")
    return df
        
df = build_ek_dataframe()
df

/tmp/ipykernel_710256/1955278187.py:28: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["t"] = pd.to_datetime(df["t"])


,task,desc
t,,
2010-01-04 00:05:32.424,SC_APP,Relative Time Sequence #170 Execution Completed.
2010-01-04 01:45:36.424,SC_APP,RTS Number 173 Started.
2010-01-04 01:45:57.425,SC_APP,Relative Time Sequence #173 Execution Completed.
2010-01-04 01:58:32.425,SC_APP,RTS Number 170 Started.
2010-01-04 01:58:38.425,SC_APP,Relative Time Sequence #170 Execution Completed.
...,...,...
2011-01-02 16:44:00.183,GRND PROC MSG,* STARTED PROCEDURE: cr_evtampdiscthn.proc RE...
2011-01-02 16:44:10.183,GRND PROC MSG,* ENDED: PROCEDURE: cr_evtampdiscthn.proc COM...
2011-01-02 16:44:15.183,GRND PROC MSG,* STARTED PROCEDURE: cr_evtampdiscthk.proc RE...


In [5]:
df["task"].unique()

array([' SC_APP', ' IM_APP', 'GRND PROC MSG', ' AC_APP'], dtype=object)

In [6]:
# with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
#     display(df[df["task"] == " IM_APP"])

In [7]:
with pd.option_context("display.max_rows", None, "display.max_colwidth", None):
    display(df[df["task"] == "GRND PROC MSG"].loc["2010-09-11":"2010-10-11"])

,task,desc
t,,
2010-09-11 18:18:56.182,GRND PROC MSG,* STARTED PROCEDURE: cr_pulsers.proc REVISION: $Rev: 14239 $
2010-09-11 18:19:19.182,GRND PROC MSG,* STARTED PROCEDURE: cr_reset.proc REVISION: $Rev: 13991 $
2010-09-11 18:19:29.182,GRND PROC MSG,* ENDED: PROCEDURE: cr_reset.proc COMPLETED SUCCESSFULLY
2010-09-11 18:19:39.182,GRND PROC MSG,* STARTED PROCEDURE: cr_detbias.proc REVISION: $Rev: 13991 $
2010-09-11 18:19:54.182,GRND PROC MSG,* ENDED: PROCEDURE: cr_detbias.proc COMPLETED SUCCESSFULLY
2010-09-11 18:19:59.182,GRND PROC MSG,* STARTED PROCEDURE: cr_evtampdiscthn.proc REVISION: $Rev: 14186 $
2010-09-11 18:20:10.182,GRND PROC MSG,* ENDED: PROCEDURE: cr_evtampdiscthn.proc COMPLETED SUCCESSFULLY
2010-09-11 18:20:14.182,GRND PROC MSG,* STARTED PROCEDURE: cr_evtampdiscthk.proc REVISION: $Rev: 14186 $
2010-09-11 18:20:24.182,GRND PROC MSG,* ENDED: PROCEDURE: cr_evtampdiscthk.proc COMPLETED SUCCESSFULLY
